This is a gradient notebook. It was used to train Yolo models by building alexyAB's or pjreddie's darknet repository. It requires CUDA and CUDNN. Some gradient notebook templates have CUDA pre-installed. This notebook provides the steps to install CUDNN additionally in gradient.

# Steps to make it work

0. Is this neccessary?
    Exchange cuda and cuda-11.6
        %cd /usr/local
        !mv cuda cuda_old
        !mv cuda-11.6 cuda
1. Download Cudnn 8.x on local machine
2. Upload zip to drive
3. Download zip here using 
        !pip install gdown
        !gdown <file>
    Make sure that the file can be shared freely through the link.
4. Move zip to /storage using,
        !mv <file> /storage/<file>
5. Move to storage, 
        %cd /storage
    and unzip using 
        !tar -xf <file>
    (The is for unzipping tar.xz files) and rm the zip file using 
        !rm <file>
6. Rename the unzipped file as cuda
7. Then run
        %cd /storage
        !dir
        !cp cuda/include/cudnn*.h /usr/local/cuda/include
        !cp cuda/lib/libcudnn* /usr/local/cuda/lib64
        !chmod a+r /usr/local/cuda/include/cudnn*.h /usr/local/cuda/lib64/libcudnn*
    This is for 'installing' cudnn in the machine. '/usr/local/cuda' is the installation path of cuda
8. Then clone darknet using, 
        %cd /notebooks
        !git clone https://github.com/pjreddie/darknet
9. Make make file changes
        %cd /notebooks/darknet
        %pycat Makefile
        !sed -i 's/GPU=0/GPU=1/g' Makefile
        !sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
        %pycat Makefile
10. Comment out first arch line(line 30) in darknet/Makefile
11. Modify the weights save freq in darknet/src/detection.c(line 138)
12. Darknet was not build for cudnn 8.x, to make it compatible replace the entire code in 'src/convolutional_layer.c' with the following
        #include "convolutional_layer.h"
        #include "utils.h"
        #include "batchnorm_layer.h"
        #include "im2col.h"
        #include "col2im.h"
        #include "blas.h"
        #include "gemm.h"
        #include <stdio.h>
        #include <time.h>

        #define PRINT_CUDNN_ALGO 0
        #define MEMORY_LIMIT 2000000000

        #ifdef AI2
        #include "xnor_layer.h"
        #endif

        void swap_binary(convolutional_layer *l)
        {
            float *swap = l->weights;
            l->weights = l->binary_weights;
            l->binary_weights = swap;

        #ifdef GPU
            swap = l->weights_gpu;
            l->weights_gpu = l->binary_weights_gpu;
            l->binary_weights_gpu = swap;
        #endif
        }

        void binarize_weights(float *weights, int n, int size, float *binary)
        {
            int i, f;
            for(f = 0; f < n; ++f){
                float mean = 0;
                for(i = 0; i < size; ++i){
                    mean += fabs(weights[f*size + i]);
                }
                mean = mean / size;
                for(i = 0; i < size; ++i){
                    binary[f*size + i] = (weights[f*size + i] > 0) ? mean : -mean;
                }
            }
        }

        void binarize_cpu(float *input, int n, float *binary)
        {
            int i;
            for(i = 0; i < n; ++i){
                binary[i] = (input[i] > 0) ? 1 : -1;
            }
        }

        void binarize_input(float *input, int n, int size, float *binary)
        {
            int i, s;
            for(s = 0; s < size; ++s){
                float mean = 0;
                for(i = 0; i < n; ++i){
                    mean += fabs(input[i*size + s]);
                }
                mean = mean / n;
                for(i = 0; i < n; ++i){
                    binary[i*size + s] = (input[i*size + s] > 0) ? mean : -mean;
                }
            }
        }

        int convolutional_out_height(convolutional_layer l)
        {
            return (l.h + 2*l.pad - l.size) / l.stride + 1;
        }

        int convolutional_out_width(convolutional_layer l)
        {
            return (l.w + 2*l.pad - l.size) / l.stride + 1;
        }

        image get_convolutional_image(convolutional_layer l)
        {
            return float_to_image(l.out_w,l.out_h,l.out_c,l.output);
        }

        image get_convolutional_delta(convolutional_layer l)
        {
            return float_to_image(l.out_w,l.out_h,l.out_c,l.delta);
        }

        static size_t get_workspace_size(layer l){
        #ifdef CUDNN
            if(gpu_index >= 0){
                size_t most = 0;
                size_t s = 0;
                cudnnGetConvolutionForwardWorkspaceSize(cudnn_handle(),
                        l.srcTensorDesc,
                        l.weightDesc,
                        l.convDesc,
                        l.dstTensorDesc,
                        l.fw_algo,
                        &s);
                if (s > most) most = s;
                cudnnGetConvolutionBackwardFilterWorkspaceSize(cudnn_handle(),
                        l.srcTensorDesc,
                        l.ddstTensorDesc,
                        l.convDesc,
                        l.dweightDesc,
                        l.bf_algo,
                        &s);
                if (s > most) most = s;
                cudnnGetConvolutionBackwardDataWorkspaceSize(cudnn_handle(),
                        l.weightDesc,
                        l.ddstTensorDesc,
                        l.convDesc,
                        l.dsrcTensorDesc,
                        l.bd_algo,
                        &s);
                if (s > most) most = s;
                return most;
            }
        #endif
            return (size_t)l.out_h*l.out_w*l.size*l.size*l.c/l.groups*sizeof(float);
        }

        #ifdef GPU
        #ifdef CUDNN
        void cudnn_convolutional_setup(layer *l)
        {
            cudnnSetTensor4dDescriptor(l->dsrcTensorDesc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT, l->batch, l->c, l->h, l->w); 
            cudnnSetTensor4dDescriptor(l->ddstTensorDesc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT, l->batch, l->out_c, l->out_h, l->out_w); 

            cudnnSetTensor4dDescriptor(l->srcTensorDesc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT, l->batch, l->c, l->h, l->w); 
            cudnnSetTensor4dDescriptor(l->dstTensorDesc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT, l->batch, l->out_c, l->out_h, l->out_w); 
            cudnnSetTensor4dDescriptor(l->normTensorDesc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT, 1, l->out_c, 1, 1); 

            cudnnSetFilter4dDescriptor(l->dweightDesc, CUDNN_DATA_FLOAT, CUDNN_TENSOR_NCHW, l->n, l->c/l->groups, l->size, l->size); 
            cudnnSetFilter4dDescriptor(l->weightDesc, CUDNN_DATA_FLOAT, CUDNN_TENSOR_NCHW, l->n, l->c/l->groups, l->size, l->size); 
            #if CUDNN_MAJOR >= 6
            cudnnSetConvolution2dDescriptor(l->convDesc, l->pad, l->pad, l->stride, l->stride, 1, 1, CUDNN_CROSS_CORRELATION, CUDNN_DATA_FLOAT);
            #else
            cudnnSetConvolution2dDescriptor(l->convDesc, l->pad, l->pad, l->stride, l->stride, 1, 1, CUDNN_CROSS_CORRELATION);
            #endif

            #if CUDNN_MAJOR >= 7
            cudnnSetConvolutionGroupCount(l->convDesc, l->groups);
            #else
            if(l->groups > 1){
                error("CUDNN < 7 doesn't support groups, please upgrade!");
            }
            #endif
            #if CUDNN_MAJOR >= 8
            int returnedAlgoCount;
            cudnnConvolutionFwdAlgoPerf_t       fw_results[2 * CUDNN_CONVOLUTION_FWD_ALGO_COUNT];
            cudnnConvolutionBwdDataAlgoPerf_t   bd_results[2 * CUDNN_CONVOLUTION_BWD_DATA_ALGO_COUNT];
            cudnnConvolutionBwdFilterAlgoPerf_t bf_results[2 * CUDNN_CONVOLUTION_BWD_FILTER_ALGO_COUNT];

            cudnnFindConvolutionForwardAlgorithm(cudnn_handle(),
                    l->srcTensorDesc,
                    l->weightDesc,
                    l->convDesc,
                    l->dstTensorDesc,
                    CUDNN_CONVOLUTION_FWD_ALGO_COUNT,
                    &returnedAlgoCount,
                fw_results);
            for(int algoIndex = 0; algoIndex < returnedAlgoCount; ++algoIndex){
                #if PRINT_CUDNN_ALGO > 0
                printf("^^^^ %s for Algo %d: %f time requiring %llu memory\n",
                       cudnnGetErrorString(fw_results[algoIndex].status),
                       fw_results[algoIndex].algo, fw_results[algoIndex].time,
                       (unsigned long long)fw_results[algoIndex].memory);
                #endif
                if( fw_results[algoIndex].memory < MEMORY_LIMIT ){
                    l->fw_algo = fw_results[algoIndex].algo;
                    break;
            }
            }

            cudnnFindConvolutionBackwardDataAlgorithm(cudnn_handle(),
                    l->weightDesc,
                    l->ddstTensorDesc,
                    l->convDesc,
                    l->dsrcTensorDesc,
                    CUDNN_CONVOLUTION_BWD_DATA_ALGO_COUNT,
                    &returnedAlgoCount,
                    bd_results);
            for(int algoIndex = 0; algoIndex < returnedAlgoCount; ++algoIndex){
                #if PRINT_CUDNN_ALGO > 0
                printf("^^^^ %s for Algo %d: %f time requiring %llu memory\n",
                       cudnnGetErrorString(bd_results[algoIndex].status),
                       bd_results[algoIndex].algo, bd_results[algoIndex].time,
                       (unsigned long long)bd_results[algoIndex].memory);
                #endif
                if( bd_results[algoIndex].memory < MEMORY_LIMIT ){
                    l->bd_algo = bd_results[algoIndex].algo;
                    break;
                }
            }

            cudnnFindConvolutionBackwardFilterAlgorithm(cudnn_handle(),
                    l->srcTensorDesc,
                    l->ddstTensorDesc,
                    l->convDesc,
                    l->dweightDesc,
                    CUDNN_CONVOLUTION_BWD_FILTER_ALGO_COUNT,
                    &returnedAlgoCount,
                    bf_results);
            for(int algoIndex = 0; algoIndex < returnedAlgoCount; ++algoIndex){
                #if PRINT_CUDNN_ALGO > 0
                printf("^^^^ %s for Algo %d: %f time requiring %llu memory\n",
                       cudnnGetErrorString(bf_results[algoIndex].status),
                       bf_results[algoIndex].algo, bf_results[algoIndex].time,
                       (unsigned long long)bf_results[algoIndex].memory);
                #endif
                if( bf_results[algoIndex].memory < MEMORY_LIMIT ){
                    l->bf_algo = bf_results[algoIndex].algo;
                    break;
                }
            }

            #else

            cudnnGetConvolutionForwardAlgorithm(cudnn_handle(),
                    l->srcTensorDesc,
                    l->weightDesc,
                    l->convDesc,
                    l->dstTensorDesc,
                    CUDNN_CONVOLUTION_FWD_SPECIFY_WORKSPACE_LIMIT,
                    2000000000,
                    &l->fw_algo);
            cudnnGetConvolutionBackwardDataAlgorithm(cudnn_handle(),
                    l->weightDesc,
                    l->ddstTensorDesc,
                    l->convDesc,
                    l->dsrcTensorDesc,
                    CUDNN_CONVOLUTION_BWD_DATA_SPECIFY_WORKSPACE_LIMIT,
                    2000000000,
                    &l->bd_algo);
            cudnnGetConvolutionBackwardFilterAlgorithm(cudnn_handle(),
                    l->srcTensorDesc,
                    l->ddstTensorDesc,
                    l->convDesc,
                    l->dweightDesc,
                    CUDNN_CONVOLUTION_BWD_FILTER_SPECIFY_WORKSPACE_LIMIT,
                    2000000000,
                    &l->bf_algo);
            #endif
        }
        #endif
        #endif

        convolutional_layer make_convolutional_layer(int batch, int h, int w, int c, int n, int groups, int size, int stride, int padding, ACTIVATION activation, int batch_normalize, int binary, int xnor, int adam)
        {
            int i;
            convolutional_layer l = {0};
            l.type = CONVOLUTIONAL;

            l.groups = groups;
            l.h = h;
            l.w = w;
            l.c = c;
            l.n = n;
            l.binary = binary;
            l.xnor = xnor;
            l.batch = batch;
            l.stride = stride;
            l.size = size;
            l.pad = padding;
            l.batch_normalize = batch_normalize;

            l.weights = calloc(c/groups*n*size*size, sizeof(float));
            l.weight_updates = calloc(c/groups*n*size*size, sizeof(float));

            l.biases = calloc(n, sizeof(float));
            l.bias_updates = calloc(n, sizeof(float));

            l.nweights = c/groups*n*size*size;
            l.nbiases = n;

            // float scale = 1./sqrt(size*size*c);
            float scale = sqrt(2./(size*size*c/l.groups));
            //printf("convscale %f\n", scale);
            //scale = .02;
            //for(i = 0; i < c*n*size*size; ++i) l.weights[i] = scale*rand_uniform(-1, 1);
            for(i = 0; i < l.nweights; ++i) l.weights[i] = scale*rand_normal();
            int out_w = convolutional_out_width(l);
            int out_h = convolutional_out_height(l);
            l.out_h = out_h;
            l.out_w = out_w;
            l.out_c = n;
            l.outputs = l.out_h * l.out_w * l.out_c;
            l.inputs = l.w * l.h * l.c;

            l.output = calloc(l.batch*l.outputs, sizeof(float));
            l.delta  = calloc(l.batch*l.outputs, sizeof(float));

            l.forward = forward_convolutional_layer;
            l.backward = backward_convolutional_layer;
            l.update = update_convolutional_layer;
            if(binary){
                l.binary_weights = calloc(l.nweights, sizeof(float));
                l.cweights = calloc(l.nweights, sizeof(char));
                l.scales = calloc(n, sizeof(float));
            }
            if(xnor){
                l.binary_weights = calloc(l.nweights, sizeof(float));
                l.binary_input = calloc(l.inputs*l.batch, sizeof(float));
            }

            if(batch_normalize){
                l.scales = calloc(n, sizeof(float));
                l.scale_updates = calloc(n, sizeof(float));
                for(i = 0; i < n; ++i){
                    l.scales[i] = 1;
                }

                l.mean = calloc(n, sizeof(float));
                l.variance = calloc(n, sizeof(float));

                l.mean_delta = calloc(n, sizeof(float));
                l.variance_delta = calloc(n, sizeof(float));

                l.rolling_mean = calloc(n, sizeof(float));
                l.rolling_variance = calloc(n, sizeof(float));
                l.x = calloc(l.batch*l.outputs, sizeof(float));
                l.x_norm = calloc(l.batch*l.outputs, sizeof(float));
            }
            if(adam){
                l.m = calloc(l.nweights, sizeof(float));
                l.v = calloc(l.nweights, sizeof(float));
                l.bias_m = calloc(n, sizeof(float));
                l.scale_m = calloc(n, sizeof(float));
                l.bias_v = calloc(n, sizeof(float));
                l.scale_v = calloc(n, sizeof(float));
            }

        #ifdef GPU
            l.forward_gpu = forward_convolutional_layer_gpu;
            l.backward_gpu = backward_convolutional_layer_gpu;
            l.update_gpu = update_convolutional_layer_gpu;

            if(gpu_index >= 0){
                if (adam) {
                    l.m_gpu = cuda_make_array(l.m, l.nweights);
                    l.v_gpu = cuda_make_array(l.v, l.nweights);
                    l.bias_m_gpu = cuda_make_array(l.bias_m, n);
                    l.bias_v_gpu = cuda_make_array(l.bias_v, n);
                    l.scale_m_gpu = cuda_make_array(l.scale_m, n);
                    l.scale_v_gpu = cuda_make_array(l.scale_v, n);
                }

                l.weights_gpu = cuda_make_array(l.weights, l.nweights);
                l.weight_updates_gpu = cuda_make_array(l.weight_updates, l.nweights);

                l.biases_gpu = cuda_make_array(l.biases, n);
                l.bias_updates_gpu = cuda_make_array(l.bias_updates, n);

                l.delta_gpu = cuda_make_array(l.delta, l.batch*out_h*out_w*n);
                l.output_gpu = cuda_make_array(l.output, l.batch*out_h*out_w*n);

                if(binary){
                    l.binary_weights_gpu = cuda_make_array(l.weights, l.nweights);
                }
                if(xnor){
                    l.binary_weights_gpu = cuda_make_array(l.weights, l.nweights);
                    l.binary_input_gpu = cuda_make_array(0, l.inputs*l.batch);
                }

                if(batch_normalize){
                    l.mean_gpu = cuda_make_array(l.mean, n);
                    l.variance_gpu = cuda_make_array(l.variance, n);

                    l.rolling_mean_gpu = cuda_make_array(l.mean, n);
                    l.rolling_variance_gpu = cuda_make_array(l.variance, n);

                    l.mean_delta_gpu = cuda_make_array(l.mean, n);
                    l.variance_delta_gpu = cuda_make_array(l.variance, n);

                    l.scales_gpu = cuda_make_array(l.scales, n);
                    l.scale_updates_gpu = cuda_make_array(l.scale_updates, n);

                    l.x_gpu = cuda_make_array(l.output, l.batch*out_h*out_w*n);
                    l.x_norm_gpu = cuda_make_array(l.output, l.batch*out_h*out_w*n);
                }
        #ifdef CUDNN
                cudnnCreateTensorDescriptor(&l.normTensorDesc);
                cudnnCreateTensorDescriptor(&l.srcTensorDesc);
                cudnnCreateTensorDescriptor(&l.dstTensorDesc);
                cudnnCreateFilterDescriptor(&l.weightDesc);
                cudnnCreateTensorDescriptor(&l.dsrcTensorDesc);
                cudnnCreateTensorDescriptor(&l.ddstTensorDesc);
                cudnnCreateFilterDescriptor(&l.dweightDesc);
                cudnnCreateConvolutionDescriptor(&l.convDesc);
                cudnn_convolutional_setup(&l);
        #endif
            }
        #endif
            l.workspace_size = get_workspace_size(l);
            l.activation = activation;

            fprintf(stderr, "conv  %5d %2d x%2d /%2d  %4d x%4d x%4d   ->  %4d x%4d x%4d  %5.3f BFLOPs\n", n, size, size, stride, w, h, c, l.out_w, l.out_h, l.out_c, (2.0 * l.n * l.size*l.size*l.c/l.groups * l.out_h*l.out_w)/1000000000.);

            return l;
        }

        void denormalize_convolutional_layer(convolutional_layer l)
        {
            int i, j;
            for(i = 0; i < l.n; ++i){
                float scale = l.scales[i]/sqrt(l.rolling_variance[i] + .00001);
                for(j = 0; j < l.c/l.groups*l.size*l.size; ++j){
                    l.weights[i*l.c/l.groups*l.size*l.size + j] *= scale;
                }
                l.biases[i] -= l.rolling_mean[i] * scale;
                l.scales[i] = 1;
                l.rolling_mean[i] = 0;
                l.rolling_variance[i] = 1;
            }
        }

        /*
        void test_convolutional_layer()
        {
            convolutional_layer l = make_convolutional_layer(1, 5, 5, 3, 2, 5, 2, 1, LEAKY, 1, 0, 0, 0);
            l.batch_normalize = 1;
            float data[] = {1,1,1,1,1,
                1,1,1,1,1,
                1,1,1,1,1,
                1,1,1,1,1,
                1,1,1,1,1,
                2,2,2,2,2,
                2,2,2,2,2,
                2,2,2,2,2,
                2,2,2,2,2,
                2,2,2,2,2,
                3,3,3,3,3,
                3,3,3,3,3,
                3,3,3,3,3,
                3,3,3,3,3,
                3,3,3,3,3};
            //net.input = data;
            //forward_convolutional_layer(l);
        }
        */

        void resize_convolutional_layer(convolutional_layer *l, int w, int h)
        {
            l->w = w;
            l->h = h;
            int out_w = convolutional_out_width(*l);
            int out_h = convolutional_out_height(*l);

            l->out_w = out_w;
            l->out_h = out_h;

            l->outputs = l->out_h * l->out_w * l->out_c;
            l->inputs = l->w * l->h * l->c;

            l->output = realloc(l->output, l->batch*l->outputs*sizeof(float));
            l->delta  = realloc(l->delta,  l->batch*l->outputs*sizeof(float));
            if(l->batch_normalize){
                l->x = realloc(l->x, l->batch*l->outputs*sizeof(float));
                l->x_norm  = realloc(l->x_norm, l->batch*l->outputs*sizeof(float));
            }

        #ifdef GPU
            cuda_free(l->delta_gpu);
            cuda_free(l->output_gpu);

            l->delta_gpu =  cuda_make_array(l->delta,  l->batch*l->outputs);
            l->output_gpu = cuda_make_array(l->output, l->batch*l->outputs);

            if(l->batch_normalize){
                cuda_free(l->x_gpu);
                cuda_free(l->x_norm_gpu);

                l->x_gpu = cuda_make_array(l->output, l->batch*l->outputs);
                l->x_norm_gpu = cuda_make_array(l->output, l->batch*l->outputs);
            }
        #ifdef CUDNN
            cudnn_convolutional_setup(l);
        #endif
        #endif
            l->workspace_size = get_workspace_size(*l);
        }

        void add_bias(float *output, float *biases, int batch, int n, int size)
        {
            int i,j,b;
            for(b = 0; b < batch; ++b){
                for(i = 0; i < n; ++i){
                    for(j = 0; j < size; ++j){
                        output[(b*n + i)*size + j] += biases[i];
                    }
                }
            }
        }

        void scale_bias(float *output, float *scales, int batch, int n, int size)
        {
            int i,j,b;
            for(b = 0; b < batch; ++b){
                for(i = 0; i < n; ++i){
                    for(j = 0; j < size; ++j){
                        output[(b*n + i)*size + j] *= scales[i];
                    }
                }
            }
        }

        void backward_bias(float *bias_updates, float *delta, int batch, int n, int size)
        {
            int i,b;
            for(b = 0; b < batch; ++b){
                for(i = 0; i < n; ++i){
                    bias_updates[i] += sum_array(delta+size*(i+b*n), size);
                }
            }
        }

        void forward_convolutional_layer(convolutional_layer l, network net)
        {
            int i, j;

            fill_cpu(l.outputs*l.batch, 0, l.output, 1);

            if(l.xnor){
                binarize_weights(l.weights, l.n, l.c/l.groups*l.size*l.size, l.binary_weights);
                swap_binary(&l);
                binarize_cpu(net.input, l.c*l.h*l.w*l.batch, l.binary_input);
                net.input = l.binary_input;
            }

            int m = l.n/l.groups;
            int k = l.size*l.size*l.c/l.groups;
            int n = l.out_w*l.out_h;
            for(i = 0; i < l.batch; ++i){
                for(j = 0; j < l.groups; ++j){
                    float *a = l.weights + j*l.nweights/l.groups;
                    float *b = net.workspace;
                    float *c = l.output + (i*l.groups + j)*n*m;
                    float *im =  net.input + (i*l.groups + j)*l.c/l.groups*l.h*l.w;

                    if (l.size == 1) {
                        b = im;
                    } else {
                        im2col_cpu(im, l.c/l.groups, l.h, l.w, l.size, l.stride, l.pad, b);
                    }
                    gemm(0,0,m,n,k,1,a,k,b,n,1,c,n);
                }
            }

            if(l.batch_normalize){
                forward_batchnorm_layer(l, net);
            } else {
                add_bias(l.output, l.biases, l.batch, l.n, l.out_h*l.out_w);
            }

            activate_array(l.output, l.outputs*l.batch, l.activation);
            if(l.binary || l.xnor) swap_binary(&l);
        }

        void backward_convolutional_layer(convolutional_layer l, network net)
        {
            int i, j;
            int m = l.n/l.groups;
            int n = l.size*l.size*l.c/l.groups;
            int k = l.out_w*l.out_h;

            gradient_array(l.output, l.outputs*l.batch, l.activation, l.delta);

            if(l.batch_normalize){
                backward_batchnorm_layer(l, net);
            } else {
                backward_bias(l.bias_updates, l.delta, l.batch, l.n, k);
            }

            for(i = 0; i < l.batch; ++i){
                for(j = 0; j < l.groups; ++j){
                    float *a = l.delta + (i*l.groups + j)*m*k;
                    float *b = net.workspace;
                    float *c = l.weight_updates + j*l.nweights/l.groups;

                    float *im  = net.input + (i*l.groups + j)*l.c/l.groups*l.h*l.w;
                    float *imd = net.delta + (i*l.groups + j)*l.c/l.groups*l.h*l.w;

                    if(l.size == 1){
                        b = im;
                    } else {
                        im2col_cpu(im, l.c/l.groups, l.h, l.w, 
                                l.size, l.stride, l.pad, b);
                    }

                    gemm(0,1,m,n,k,1,a,k,b,k,1,c,n);

                    if (net.delta) {
                        a = l.weights + j*l.nweights/l.groups;
                        b = l.delta + (i*l.groups + j)*m*k;
                        c = net.workspace;
                        if (l.size == 1) {
                            c = imd;
                        }

                        gemm(1,0,n,k,m,1,a,n,b,k,0,c,k);

                        if (l.size != 1) {
                            col2im_cpu(net.workspace, l.c/l.groups, l.h, l.w, l.size, l.stride, l.pad, imd);
                        }
                    }
                }
            }
        }

        void update_convolutional_layer(convolutional_layer l, update_args a)
        {
            float learning_rate = a.learning_rate*l.learning_rate_scale;
            float momentum = a.momentum;
            float decay = a.decay;
            int batch = a.batch;

            axpy_cpu(l.n, learning_rate/batch, l.bias_updates, 1, l.biases, 1);
            scal_cpu(l.n, momentum, l.bias_updates, 1);

            if(l.scales){
                axpy_cpu(l.n, learning_rate/batch, l.scale_updates, 1, l.scales, 1);
                scal_cpu(l.n, momentum, l.scale_updates, 1);
            }

            axpy_cpu(l.nweights, -decay*batch, l.weights, 1, l.weight_updates, 1);
            axpy_cpu(l.nweights, learning_rate/batch, l.weight_updates, 1, l.weights, 1);
            scal_cpu(l.nweights, momentum, l.weight_updates, 1);
        }

        image get_convolutional_weight(convolutional_layer l, int i)
        {
            int h = l.size;
            int w = l.size;
            int c = l.c/l.groups;
            return float_to_image(w,h,c,l.weights+i*h*w*c);
        }

        void rgbgr_weights(convolutional_layer l)
        {
            int i;
            for(i = 0; i < l.n; ++i){
                image im = get_convolutional_weight(l, i);
                if (im.c == 3) {
                    rgbgr_image(im);
                }
            }
        }

        void rescale_weights(convolutional_layer l, float scale, float trans)
        {
            int i;
            for(i = 0; i < l.n; ++i){
                image im = get_convolutional_weight(l, i);
                if (im.c == 3) {
                    scale_image(im, scale);
                    float sum = sum_array(im.data, im.w*im.h*im.c);
                    l.biases[i] += sum*trans;
                }
            }
        }

        image *get_weights(convolutional_layer l)
        {
            image *weights = calloc(l.n, sizeof(image));
            int i;
            for(i = 0; i < l.n; ++i){
                weights[i] = copy_image(get_convolutional_weight(l, i));
                normalize_image(weights[i]);
                /*
                   char buff[256];
                   sprintf(buff, "filter%d", i);
                   save_image(weights[i], buff);
                 */
            }
            //error("hey");
            return weights;
        }

        image *visualize_convolutional_layer(convolutional_layer l, char *window, image *prev_weights)
        {
            image *single_weights = get_weights(l);
            show_images(single_weights, l.n, window);

            image delta = get_convolutional_image(l);
            image dc = collapse_image_layers(delta, 1);
            char buff[256];
            sprintf(buff, "%s: Output", window);
            //show_image(dc, buff);
            //save_image(dc, buff);
            free_image(dc);
            return single_weights;
        }

13. The above was found at link, https://github.com/arnoldfychen/darknet/blob/master/src/convolutional_layer.c
14. A parent link for the above link https://github.com/pjreddie/darknet/issues/2257

# DO THESE AGAIN ONLY IF NECCESSARY

## Building darknet

In [10]:
# Move yolo_builds if present before modifying darknet build
!mv /notebooks/darknet/Yolo_builds /notebooks

#Remove old darknet if present. It will delete Yolo_build files also
!rm -r /notebooks/darknet

# Clone darknet
%cd /notebooks
#!git clone https://github.com/AlexeyAB/darknet
!git clone https://github.com/pjreddie/darknet

#Make makefile changes
%cd /notebooks/darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

#Modify weights save freq here

# Move yolo_builds back
!mv /notebooks/Yolo_builds /notebooks/darknet

/notebooks
Cloning into 'darknet'...
remote: Enumerating objects: 5952, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 5952 (delta 2), reused 0 (delta 0), pack-reused 5946
Receiving objects: 100% (5952/5952), 6.37 MiB | 15.24 MiB/s, done.
Resolving deltas: 100% (3930/3930), done.
Updating files: 100% (979/979), done.
/notebooks/darknet


## Compile and check

In [18]:
%cd /notebooks/darknet
!make
!./darknet

/notebooks/darknet
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
  232 |         cudaThreadSynchronize();
      |         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:1054:57: note: declared here
 1054 | extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
      |                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN 

## Create persistent .zip model, dataset, weight files in storage under /storage/<model_name>

In [6]:
%cd /storage/Colour_Helmet_detection
!dir

/storage/Colour_Helmet_detection
Custom_1500_101.zip  Custom_1500_101_data.zip


In [3]:
%cd /storage

# Remove old model zip if it exists and make new
!rm -r Colour_Helmet_detection
!mkdir Colour_Helmet_detection

# Download data_set, model_files, weights as zip files
%cd /storage/Colour_Helmet_detection
!pip install gdown
!gdown --id 193uz8mZsLgIyx3hzYSjyc0TyEJ55CutK #data_set
!gdown --id 1sClbYHkSM1GkE6G4YymAFVWGUiJpHLgy #model_files
#!gdown --id 1-QQTVgHSPIea1RX8IV109PmX40-ZZNR6 #weights

# List all files present under the model
!ls -lh

/storage
/storage/Colour_Helmet_detection
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=002a3ca013129ebf618f8a29d06ec11ad4161498b2fb04d7f2f3375d5957a6d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-iioslyd7/wheels/8d/df/71/846b2aa0fabaac2af23fbc5214eeaa55f0616e9d1a05187d72
Successfully built gdown
/opt/conda/lib/python3.8/site-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=193uz8mZsLgIyx3hzYSjyc0TyEJ55CutK
To: /storage/Colour_Helmet_detection/Custom_1500_101_data.zip
100%|████████████████████████████████████████| 420M/420M [00:21<00:00, 19.3MB/s]
/opt/conda/lib/p

# YOU MAY DO THIS EVERYTIME, THOUGH NOT NECCESSARY

## Create the folder tree 
        
        Yolo_builds        
        |
        |
        |---<name>     
            |
            |---<name>.data - (The .data file for training)				
            |
            |---class.txt - (.txt file containing names of classes)
            |
            |---yolovx.cfg - (Yolo configuration file)
            |
            |---train.txt - (txt file containing the training images)
            |
            |---test.txt - (txt file containing the test images)
            |
            |---valid.txt - (txt file containing the validation images)	            
            |
            |---Data_set
            |   |      
            |   |---Images and annotations
            |       |
            |    	|---(all training images and their annotations)            
            |               
            |---Weights_save
                |
                |---(pre-trained weight files)            

In [4]:
# Move to darknet
%cd /notebooks/darknet

# Remove old Yolo_builds and make new
!rm -r Yolo_builds
!mkdir Yolo_builds

# Move to Yolo_builds and create <model_name> folder
%cd Yolo_builds
!mkdir Colour_Helmet_detection

# Move to Yolo_builds/<model_name> and create the 'Data_set' and 'Weights_save' folder
%cd Colour_Helmet_detection
!mkdir Data_set
!mkdir Weights_save

#Move to Yolo_builds/<model_name>/Data_set and create 'Images_and_annotations'
%cd Data_set
!mkdir Images_and_annotations

/notebooks/darknet
/notebooks/darknet/Yolo_builds
/notebooks/darknet/Yolo_builds/Colour_Helmet_detection
/notebooks/darknet/Yolo_builds/Colour_Helmet_detection/Data_set


## Unzip the persistent files in /storage/model_name and store them in  /darknet/Yolo_builds/model_name

In [7]:
# Unzip model files to Yolo_builds/<model_name> foler 
!unzip /storage/Colour_Helmet_detection/Custom_1500_101.zip -d /notebooks/darknet/Yolo_builds/Colour_Helmet_detection

# Unzip dataset to Yolo_builds/<model_name>/Data_set/Images_and_annotations
!unzip /storage/Colour_Helmet_detection/Custom_1500_101_data.zip -d /notebooks/darknet/Yolo_builds/Colour_Helmet_detection/Data_set

# Move weights to Yolo_builds/<model_name>/Weights_save
#!mv /storage/Colour_Helmet_detection/yolov3-voc_3100.weights /notebooks/darknet/Yolo_builds/Colour_Helmet_detection/Weights_save/yolov3-voc_3100.weights

Archive:  /storage/Colour_Helmet_detection/Custom_1500_101.zip
 extracting: /notebooks/darknet/Yolo_builds/Colour_Helmet_detection/class_names.txt  
  inflating: /notebooks/darknet/Yolo_builds/Colour_Helmet_detection/Colour_Helmet_detection.data  
  inflating: /notebooks/darknet/Yolo_builds/Colour_Helmet_detection/train_data.txt  
  inflating: /notebooks/darknet/Yolo_builds/Colour_Helmet_detection/valid_data.txt  
  inflating: /notebooks/darknet/Yolo_builds/Colour_Helmet_detection/yolov3-voc.cfg  
Archive:  /storage/Colour_Helmet_detection/Custom_1500_101_data.zip
  inflating: /notebooks/darknet/Yolo_builds/Colour_Helmet_detection/Data_set/Images_and_annotations/hard_hat_workers0.png  
  inflating: /notebooks/darknet/Yolo_builds/Colour_Helmet_detection/Data_set/Images_and_annotations/hard_hat_workers0.txt  
  inflating: /notebooks/darknet/Yolo_builds/Colour_Helmet_detection/Data_set/Images_and_annotations/hard_hat_workers1.png  
  inflating: /notebooks/darknet/Yolo_builds/Colour_Helmet

# YOU MUST DO THESE EVERYTIME

## Convert the model files to Unix format

Do it if you modify the model files
Make the .conf and .data file changes before converting

In [1]:
%cd /notebooks/darknet/Yolo_builds/Colour_Helmet_detection

!apt install dos2unix
!dos2unix train_data.txt
!dos2unix valid_data.txt
!dos2unix Colour_Helmet_detection.data
!dos2unix class_names.txt
!dos2unix yolov3-voc.cfg

/notebooks/darknet/Yolo_builds/Colour_Helmet_detection
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 28 not upgraded.
Need to get 374 kB of archives.
After this operation, 1342 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 dos2unix amd64 7.4.0-2 [374 kB]
Fetched 374 kB in 0s (4923 kB/s) 
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline

78Selecting previously unselected package dos2unix.
(Reading database ... 20930 files and directories currently installed.)
Preparing to unpack .../dos2unix_7.4.0-2_amd64.deb ...
7Progress: [  0%] [.........................................

## Train the model

        %cd /notebooks/darknet
        !./darknet detector train Yolo_builds/Colour_Helmet_detection/Colour_Helmet_detection.data Yolo_builds/Colour_Helmet_detection/yolov3-voc.cfg /notebooks/yolov3-voc.backup


In [1]:
%cd /notebooks/darknet
!./darknet detector train Yolo_builds/Colour_Helmet_detection/Colour_Helmet_detection.data Yolo_builds/Colour_Helmet_detection/yolov3-voc.cfg /notebooks/yolov3-voc.backup

/notebooks/darknet
yolov3-voc
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32  0.299 BFLOPs
    1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32  0.177 BFLOPs
    3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    4 res    1                 208 x 208 x  64   ->   208 x 208 x  64
    5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
    7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    8 res    5                 104 x 104 x 128   ->   104 x 104 x 128
    9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
   10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
   11 r

# OTHER THINGS

## Check storage

In [7]:
%cd /notebooks
!rm -r .Trash-0
!du -h --max-depth=1 #This command will list all directories within the current directory recursively till depth=1 along with their sizes

/notebooks
100K	./.git
419M	./darknet
512	./.ipynb_checkpoints
1.4G	.


# Gdrive

## Installation here
1. Download gdrive here from version available in https://github.com/prasmussen/gdrive/releases
2. Upload the tar-file to /storage/gdrive_folder
3. Extract

        %cd /storage
        !wget https://github.com/prasmussen/gdrive/releases/download/2.1.1/gdrive_2.1.1_linux_386.tar.gz

        !mkdir gdrive_folder
        !mv gdrive_2.1.1_linux_386.tar.gz gdrive_folder/gdrive.tar.gz
        !tar -xf gdrive.tar.gz

The persistant structure is 
    /Storage
    |
    |---gdrive_folder
        |
        |--gdrive.exe
        |
        |--*gdrive.tar.gz

## Usage

If you have already previously created a token file in the same machine and gdrive is also installed then you can skip steps 1, 2, 4.

1. Download gdrive on the local machine from https://github.com/prasmussen/gdrive/releases
2. Extract the tar file using 
        tar -xf <file_name>
3. Then run the extract gdrive using
        ./gdrive about
4. Get the authorization code from the output-link. This will create a token file at C:/users/<user_name>/AppData/Roaming/.gdrive
5. Upload the token from C:/users/<user_name>/AppData/Roaming/.gdrive here to /notebooks

Then run all gdrive commands as 

        %cd /storage/gdrive_folder
        !GDRIVE_CONFIG_DIR="/notebooks" ./gdrive 
        #/notebooks is the parent folder of the token file
        
## About the token file

1. The token file allows a time limited access to the authorized google drive
2. Anyone can full-access the authorized drive using the token file hence for safety reasons DELETE THE TOKEN AFTER USE IMMEDIATELY

In [5]:
%cd /storage/gdrive_folder
!GDRIVE_CONFIG_DIR="/notebooks" ./gdrive list
!GDRIVE_CONFIG_DIR="/notebooks" ./gdrive upload /notebooks/yolov3-voc_26000.weights

/storage/gdrive_folder
Id                                             Name                                       Type   Size       Created
11RRS-N8cTm3LFCoTDr071Ip1DqFCQvvZ              yolov3-voc_25000.weights                   bin    246.4 MB   2022-07-17 16:21:29
1L0drBXlqGLdQRwdpIk-JE1wb0BNscEIt              Image_detector_using_openCV.ipynb          bin    13.2 KB    2022-06-17 03:36:16
1Rlf8YbASV56dwxQyUAJ_PJNH4J8K7qg_              yolov3-voc_27200.weights                   bin    246.4 MB   2022-07-17 16:19:55
1wwIYkk3bkriqcDokcAEgMkj8lgnXH9Gk              yolov3-voc_24000.weights                   bin    246.4 MB   2022-07-17 16:18:12
1lS1yx845ERa21EUetjyrTOFXsHSLH-Qu              YoloV3_training_for...et_detection.ipynb   bin    549.7 KB   2022-06-19 06:42:23
1ZwKXEm7rf2YgxcEn61o4Cs0RZN2cs052              yolov3-voc_23000.weights                   bin    246.4 MB   2022-07-16 09:47:16
1i-DmuIggw-a9xUSBtCXfZZu5RG943T8M              YoloV4_training_for...et_detection.ipynb   bin